<a href="https://colab.research.google.com/github/Cmantilla866/LeaguePredictor/blob/master/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# main libraries
import pandas as pd
import numpy as np
import time
# visual libraries
from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')
# sklearn libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef,classification_report,roc_curve,roc_auc_score
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

In [0]:
teams = pd.read_csv('Teams.csv')
games = pd.read_csv('Games.csv')

In [190]:
teams.head()

,Name,Win Rate,Kd,Gpm,Gdm,Kills Game,Deaths Game,Towers Killed,Towers Lost,Fb,Ft,Drapg,Dra,Herpg,Her,Dra 15,Td 15,Gd 15,Nash Pg,Nash,Csm,Dpm,Wpm,Vwpm,Wcpm
0,CNB e-Sports Club,50.0,0.90,1767,-33,10.5,11.7,6.2,7.1,55.0,50.0,2.30,47.9,0.60,60.0,1.10,0.3,311,0.80,47.5,32.6,1741,3.5,1.48,1.45
1,Flamengo eSports,95.0,1.77,1912,256,15.2,8.5,9.5,2.6,75.0,80.0,3.40,77.8,0.75,75.0,1.35,0.6,1707,1.15,89.5,33.6,1919,3.8,1.35,1.43
2,INTZ eSports,65.0,1.33,1848,94,13.4,10.1,7.4,5.9,60.0,75.0,2.15,51.7,0.55,57.9,1.00,0.5,937,0.85,50.0,33.0,1769,3.4,1.44,1.39
3,KaBuM! e-Sports,40.0,0.82,1780,-51,12.0,14.7,6.3,6.9,40.0,50.0,2.35,45.8,0.50,50.0,0.60,0.1,-324,0.60,33.3,32.8,1932,3.5,1.48,1.36
4,ProGaming eSports,10.0,0.60,1637,-263,7.6,12.6,2.8,9.4,50.0,15.0,0.65,15.6,0.30,30.0,0.25,-1.2,-2523,0.20,11.4,31.7,1509,3.4,1.40,1.19


In [191]:
games.head()

,Blue Side,Score,Red Side
0,Vivo Keyd,1,Flamengo eSports
1,CNB e-Sports Club,0,ProGaming eSports
2,Redemption POA,1,Uppercut eSports
3,KaBuM! e-Sports,1,INTZ eSports
4,ProGaming eSports,1,Vivo Keyd


In [0]:
df1 = pd.merge(games, teams, how='inner', left_on = 'Red Side', right_on = 'Name')
df = pd.merge(teams,df1,how='inner',left_on='Name',right_on='Blue Side')
df.to_csv('dataframe.csv')

In [193]:
frame = pd.read_csv('dataframe.csv')
df.isnull().values.any()

False

In [0]:
frame =  frame.drop(['Unnamed: 0','Name_x','Name_y'],axis=1)

In [0]:
aparte1 = frame[['Blue Side','Red Side','Score']]
frame = frame.drop(['Blue Side','Red Side','Score'],axis=1)

In [0]:
std = StandardScaler()
f = std.fit_transform(frame)

In [0]:
frame = pd.DataFrame(f,columns=frame.columns.values)

In [0]:
frame = pd.concat([aparte1,frame], axis = 1)


In [199]:
frame.head()

,Blue Side,Red Side,Score,Win Rate_x,Kd_x,Gpm_x,Gdm_x,Kills Game_x,Deaths Game_x,Towers Killed_x,Towers Lost_x,Fb_x,Ft_x,Drapg_x,Dra_x,Herpg_x,Her_x,Dra 15_x,Td 15_x,Gd 15_x,Nash Pg_x,Nash_x,Csm_x,Dpm_x,Wpm_x,Vwpm_x,Wcpm_x,Win Rate_y,Kd_y,Gpm_y,Gdm_y,Kills Game_y,Deaths Game_y,Towers Killed_y,Towers Lost_y,Fb_y,Ft_y,Drapg_y,Dra_y,Herpg_y,Her_y,Dra 15_y,Td 15_y,Gd 15_y,Nash Pg_y,Nash_y,Csm_y,Dpm_y,Wpm_y,Vwpm_y,Wcpm_y
0,CNB e-Sports Club,Flamengo eSports,1,-0.032187,-0.435957,-0.249499,-0.205633,-0.546914,-0.057469,0.063128,0.621802,0.387579,-0.045728,0.357956,-0.164243,0.759235,0.681649,1.34414,0.423073,0.176662,0.416415,-0.140279,-0.028721,-0.202246,0.483915,1.140994,0.513479,1.892252,1.445868,1.103814,1.234623,1.081210,-1.130132,1.804138,-1.702418,1.822431,1.507893,2.245748,1.781478,1.782021,1.708798,2.363488,0.823528,1.016666,1.693888,1.897849,0.551203,0.712939,1.330955,0.344283,0.405850
1,CNB e-Sports Club,ProGaming eSports,0,-0.032187,-0.435957,-0.249499,-0.205633,-0.546914,-0.057469,0.063128,0.621802,0.387579,-0.045728,0.357956,-0.164243,0.759235,0.681649,1.34414,0.423073,0.176662,0.416415,-0.140279,-0.028721,-0.202246,0.483915,1.140994,0.513479,-1.931182,-1.170158,-1.617174,-1.505603,-1.600080,0.362896,-1.894972,1.937427,-0.008019,-1.928735,-2.615497,-2.456961,-1.411777,-1.455395,-2.213881,-1.927022,-1.718478,-2.090012,-2.091056,-0.647653,-1.448985,0.209026,0.652926,-0.888965
2,CNB e-Sports Club,ProGaming eSports,1,-0.032187,-0.435957,-0.249499,-0.205633,-0.546914,-0.057469,0.063128,0.621802,0.387579,-0.045728,0.357956,-0.164243,0.759235,0.681649,1.34414,0.423073,0.176662,0.416415,-0.140279,-0.028721,-0.202246,0.483915,1.140994,0.513479,-1.931182,-1.170158,-1.617174,-1.505603,-1.600080,0.362896,-1.894972,1.937427,-0.008019,-1.928735,-2.615497,-2.456961,-1.411777,-1.455395,-2.213881,-1.927022,-1.718478,-2.090012,-2.091056,-0.647653,-1.448985,0.209026,0.652926,-0.888965
3,CNB e-Sports Club,Uppercut eSports,0,-0.032187,-0.435957,-0.249499,-0.205633,-0.546914,-0.057469,0.063128,0.621802,0.387579,-0.045728,0.357956,-0.164243,0.759235,0.681649,1.34414,0.423073,0.176662,0.416415,-0.140279,-0.028721,-0.202246,0.483915,1.140994,0.513479,-0.356827,-0.432304,0.045102,-0.000855,-0.329995,0.144404,-0.073023,0.171032,-1.106289,0.186113,0.478022,0.520849,-1.056910,-1.103818,0.074803,0.365103,0.007960,0.698125,0.283900,0.298813,-0.383842,-0.351938,-0.396459,-0.079706
4,CNB e-Sports Club,Uppercut eSports,1,-0.032187,-0.435957,-0.249499,-0.205633,-0.546914,-0.057469,0.063128,0.621802,0.387579,-0.045728,0.357956,-0.164243,0.759235,0.681649,1.34414,0.423073,0.176662,0.416415,-0.140279,-0.028721,-0.202246,0.483915,1.140994,0.513479,-0.356827,-0.432304,0.045102,-0.000855,-0.329995,0.144404,-0.073023,0.171032,-1.106289,0.186113,0.478022,0.520849,-1.056910,-1.103818,0.074803,0.365103,0.007960,0.698125,0.283900,0.298813,-0.383842,-0.351938,-0.396459,-0.079706


In [0]:
frame = pd.get_dummies(frame, columns=['Blue Side', 'Red Side'])

In [0]:
train_x, test_x, train_y, test_y = train_test_split(frame.drop('Score', axis=1), frame['Score'], test_size=0.2, random_state=42)

In [202]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=13)
model.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=13, verbose=0,
                       warm_start=False)

In [203]:
predicted = model.predict(test_x)
model.score(test_x, test_y)

0.7738095238095238

In [0]:
probabilities = model.predict_proba(test_x)

In [205]:
roc_auc_score(test_y, probabilities[:, 1])

0.8698620395391836

In [206]:
confusion_matrix(test_y, predicted)

array([[68, 11],
       [27, 62]])

In [207]:
train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

0.9745222929936306

In [208]:
recall_score(train_y, train_predictions)

0.9776357827476039

In [209]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(train_x, train_y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [210]:
predicted = clf.predict(test_x)
clf.score(test_x, test_y)

0.7380952380952381

In [211]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_x, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [212]:
predicted = neigh.predict(test_x)
clf.score(test_x, test_y)

0.7380952380952381

In [213]:
probabilities = neigh.predict_proba(test_x)
roc_auc_score(test_y, probabilities[:, 1])

0.8197980372635472

In [214]:
confusion_matrix(test_y, predicted)

array([[64, 15],
       [28, 61]])

In [216]:
joblib.dump(model,'modelo.pkl')

['modelo.pkl']